# Part II - Prosper Loan Data Analysis
## by Fidel Kalele Wafula

## Investigation Overview
The investigation of the Prosper Loans Data particularly aimed to look into how the borrower interest rate and the Prosper's loan credit rating interacts with other loan features, as well as explore the loan disbursement trends over time in order to inform the institution on the effective  business growth startegies.


## Dataset Overview
The Prosper Loans dataset had 113,937 loans with 81 features, such as, credit grade, loan status, borrower APR among others. After performing preliminary data wrangling on the original data, the resulting clean data set had 83,981 rows and 18 columns. These variables are majorly either numeric or categorical.

In [ ]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

# suppress warnings from final output
import warnings
warnings.simplefilter("ignore")

In [ ]:
# load in the dataset into a pandas dataframe
loans = pd.read_csv('prosperLoanData.csv')

# Subset the dataset to extract variables of interest:
columns = ['ListingCreationDate','Term','LoanStatus','BorrowerAPR','BorrowerRate','ListingCategory (numeric)',
           'Occupation','EmploymentStatus','DebtToIncomeRatio','IncomeRange','StatedMonthlyIncome','MonthlyLoanPayment',
           'LoanOriginalAmount','LoanOriginationDate','ProsperRating (Alpha)','IsBorrowerHomeowner']
loans_sub = loans[columns]

# Extract only the non-duplicated records:
loans_sub = loans_sub.loc[~loans_sub.duplicated(),:]

# Fill null entries on the Occupation variable as 'Unspecified':
loans_sub['Occupation'] = loans_sub['Occupation'].fillna('Unspecified')

# Eliminate null entries in the Prosper Rating (Alpha) variable:
loans_sub = loans_sub.loc[loans_sub['ProsperRating (Alpha)'].notnull()]

# Amend Prosper Rating and Income ranges to ordered categories:

# Store the correct variable orders in a dictionary:
ordered_dict = {'ProsperRating (Alpha)': ['AA', 'A', 'B', 'C', 'D', 'E','HR'],
                'IncomeRange': ['$0', '$1-24,999', '$25,000-49,999', 
                              '$50,000-74,999', '$75,000-99,999', '$100,000+']}

# Use for loop to assign each column to the proper order
for key, value in ordered_dict.items():
    ordered_cat = pd.api.types.CategoricalDtype(categories=value, ordered=True)
    loans_sub[key] = loans_sub[key].astype(ordered_cat)

# Mapping listing category numeric value to its description:
numeric_list = {0 : 'Not Available', 1 : 'Debt Consolidation', 2 : 'Home Improvement', 3: 'Business', 
             4 : 'Personal Loan', 5 : 'Student Use', 6 : 'Auto', 7 : 'Other', 8 : 'Baby&Adoption', 
             9 : 'Boat', 10 : 'Cosmetic Procedure', 11 : 'Engagement Ring', 12 : 'Green Loans', 
             13 : 'Household Expenses', 14 : 'Large Purchases', 15 : 'Medical/Dental', 16 : 'Motorcycle', 
             17 : 'RV', 18 : 'Taxes', 19 : 'Vacation', 20 : 'Wedding Loans'}

loans_sub['ListingCategory'] = loans_sub['ListingCategory (numeric)'].map(numeric_list)

# Delete numeric losting categories to retain only the descriptive listings column:
loans_sub.drop(['ListingCategory (numeric)'], axis=1, inplace=True)

# Amend the data types of datetime variables to appropriately:
loans_sub['LoanOriginationDate'] = pd.to_datetime(loans_sub['LoanOriginationDate'])
loans_sub['ListingCreationDate'] = pd.to_datetime(loans_sub['ListingCreationDate'])

# Extract date information from ListingCreationDate column
loans_sub['ListingDate'] = loans_sub['ListingCreationDate'].dt.date

# Extract year information from ListingCreationDate column
loans_sub['ListingYear'] = loans_sub['ListingCreationDate'].dt.year

# Extract month information from ListingCreationDate column
loans_sub['ListingMonth'] = loans_sub['ListingCreationDate'].dt.month

# Convert 'ListingDate' type from str to datetime 
loans_sub['ListingDate'] = pd.to_datetime(loans_sub['ListingDate'])

# Drop unused column
loans_sub.drop(['ListingCreationDate'], axis=1, inplace=True)

# Convert 'ListingMonth' type  from int64 to str:
loans_sub['ListingMonth'] = loans_sub['ListingMonth'].astype(str)

# Replace the numerical representation of months to text of the format 'mmm':
loans_sub['ListingMonth'] = loans_sub['ListingMonth'].replace(['1', '2','3','4','5','6','7','8', '9','10','11','12'], 
                                                              ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sept','Oct','Nov','Dec'])

# Amend the data type of listing month to ordered categorical:
ordered_months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sept','Oct','Nov','Dec']
months_class = pd.api.types.CategoricalDtype(ordered = True, categories = ordered_months)
loans_sub['ListingMonth'] = loans_sub['ListingMonth'].astype(months_class)

# Convert Listing Year data type from integer to string:
loans_sub['ListingYear'] = loans_sub['ListingYear'].astype(str)

# Drop duplicates:
loans_sub = loans_sub.drop_duplicates()

# Drop null entry records:
loans_sub = loans_sub[~loans_sub.isnull()]

## Overall Distribution of Customers' Prosper Loan Credit Ratings
The Prosper Credit Ratings are arranged from 'AA' (lowest risk, lowest return) to 'HR' (highest risk, highest return). The credit rating 'C' is the rating with the highest number of borrowers (21.6%), while 'AA' has the least number (6.3%).

In [ ]:
# Calculate counts of total loans and total loans per prosper rating:
counts = loans_sub['ProsperRating (Alpha)'].value_counts()
n_loans = loans_sub['ProsperRating (Alpha)'].value_counts().sum()

# Plot the the distribution of the loan customers across the prosper rating:
plt.figure(figsize=[8,6])
base_color = sb.color_palette()[0]
sb.barplot(loans_sub['ProsperRating (Alpha)'].value_counts().index.values, counts, color = base_color);
plt.ylim(0, 20000)
plt.xlabel('Prosper Credit Rating');
plt.ylabel('Count of Borrowers');
plt.title('Distribution of Loan Borrowers across Prosper Ratings:');

# Loop through the prosper rating categories to compute and display percentage:
for i in range(counts.shape[0]):
    count = counts[i]
    pct_string = '{:0.1f}%'.format(100*count/n_loans)
    plt.text(i-0.1,count+700, pct_string, va = 'center');
plt.show()

## Prosper's Loan Book Performance
The majority of the loans in the Prosper's loan book are paying well (i.e. either in the current status or completed status). All loans that were past due were aggregated into one category, 'Past Due' which consituted only 2.4% of the loan book. The remaining categories of loans comprised of loans that were either charged off or defaulted (7.5%).

In [ ]:
# Use lambda function to lump all past due categories into ine category, 'Past Due':
loans_sub['LoanStatus'] = loans_sub['LoanStatus'].apply(lambda x: x if 'Past Due' not in x else 'Past Due')

n_counts = loans_sub['LoanStatus'].value_counts().sum()
counts = loans_sub['LoanStatus'].value_counts()

# Plot the count of borrwers in the different loan status:
plt.figure(figsize=[8,6])
sb.barplot(loans_sub['LoanStatus'].value_counts().index, counts, color=base_color)
plt.xticks(rotation=45);
plt.xlabel('Loan Status');
plt.ylabel('Count of Borrowers')
plt.title('Count of customers per Loan Status type');

# Use for loop to label each bar with its equivalent percentage:
for i in range(counts.shape[0]):
    count = counts[i]
    pct_string = '{:0.1f}%'.format(100*count/n_loans)
    plt.text(i-0.1,count+700, pct_string, va = 'center');

plt.show()

## Propser Loans Disbursements Trends
The loan disbursements amounts are highest at the beginning and end of the year. Towards mid-year, the disbursement values decrease. However, there are distinct cases of high loan amounts applied depicted by the outliers on the below box plots of the months between April and August.

In [ ]:
# Plot relationship between total loan amounts and the listing month:
plt.figure(figsize=[10,6])
ax = sb.boxplot(data=loans_sub, x='ListingMonth', y='LoanOriginalAmount', color=base_color);
plt.xlabel('Listing Month');
plt.ylabel('Total Loan Amount ($)');
plt.title('Relationship between Total Loan Amounts applied and Listing Month');
plt.show()

## Relationship between the Borrower Rate, Loan Term and Prosper Ratings
The borrower rate generally is lower for best rated loans while it increases as the loan ratings deteroriates. Across most of the rating categories, borrower rate increases with the increasing loan term, except for worst rating (HR) whose only term is medium term (36 months).

In [ ]:
plt.figure(figsize=(10,6))
ax = sb.pointplot(data = loans_sub, x = 'ProsperRating (Alpha)', y = 'BorrowerRate', 
                  hue = 'Term', palette = 'Greens', linestyles = '', dodge = 0.4, ci = 'sd')
plt.xlabel('Prosper Rating');
plt.ylabel('Borrower Interest Rate');
plt.title('Relationship between the Borrower Rate, Loan Term and Prosper Ratings');
plt.yticks(np.arange(.05, .4, .05), ['5%', '10%', '15%', '20%', '25%', '30%', '35%', '40%']);
plt.show()

## Borrower APR and Loan Status for various Employment Status
Customers whose loans are status are complete, current or having final payment in progress are cheaper as they enjoy lower APR. Furthermore, employed, full-time, retired and part-time employees exhibit good performance on loans as compared to other employment statuses. Subsequently, these particular categories of employees enjoy cheaper loans wih least APR.

In [ ]:
# Create list for the performing loans:
status_performing = ['Completed', 'Current', 'FinalPaymentInProgress']

# A function to categorize each loan into performing/non-performing type:
def status_cat (entry):
    '''A function to categorize loan status'''
    for status in status_performing:
        if status in entry:
            return 'Performing'
    return 'Non-performing'

# Apply status_cat function to classify loan status:
loans_sub['StatusCategory'] = loans_sub.LoanStatus.apply(lambda x: status_cat(x))

# Plot box plots of the borrower APR for the oerforming and non-performing loans per different employment status:
plt.figure(figsize=[13, 6])
sb.boxplot(data=loans_sub, x='EmploymentStatus', y='BorrowerAPR', hue='StatusCategory', dodge=0.3, color=base_color);
plt.yticks(np.arange(.05, .5, .05), ['5%', '10%', '15%', '20%', '25%', '30%', '35%', '40%', '45%', '50%']);
plt.title('Borrower APR and Loan Status for various Employment Status');
plt.xlabel('Employment Status');
plt.ylabel('Borrower APR');
plt.xticks(rotation=45);
plt.legend(loc=6, bbox_to_anchor=(1.0,0.5));

## Borrower APR and Loan Original Amount against Prosper Rating with Loan Term
There seems to be a negative correlation between Prosper rating and the borrower APR. Loans with the best ratings (AA, A) have the least APR, whereas the loans with bad ratings (E, HR) have the most APR. This is a balance control that rewards the best loan customers and motivates the high risk customers. Also, there is a considerable difference in the APRs of the best rated loans (A, AA) whose short and medium term loans have the least APRs.

Customers with the best ratings are the most favorable borrowers of the long-term loans of higher amounts than the ones with poor ratings. This is still the case even for the medium- and short-term loans.

In [ ]:
fig = plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 2)
sb.pointplot(data=loans_sub, x='ProsperRating (Alpha)', y='LoanOriginalAmount', hue='Term', scale=.7, 
             errwidth=.8, palette='BuPu');
plt.yticks(np.arange(4000, 20000, 4000), ['$ 4k', '$ 8k', '$ 12k', '$ 16k', '$ 20k']);
plt.xlabel('Prosper Rating');
plt.ylabel('Borrower APR');
plt.title('Borrower APR vs Prosper Loan Rating');

plt.subplot(1, 2, 1)
sb.pointplot(data=loans_sub, x='ProsperRating (Alpha)', y='BorrowerAPR', hue='Term', scale=.7, linestyles='', palette='BuPu');
plt.yticks(np.arange(0.05, 0.4, 0.05), ['5%', '10%', '15%', '20%', '25%', '30%', '35%', '40%']);
plt.xlabel('Prosper Rating');
plt.ylabel('Loan Original Amount');
plt.title('Loan Original Amount vs Prosper Loan Rating');


In [ ]:
# The command below will generate the HTML slideshow. 
!jupyter nbconvert Part_II_notebook.ipynb --to slides --post serve --no-input --no-prompt 